#### imports

In [84]:
import csv
import requests
import math
import pprint
import os
import json
import pandas as pd
from pandas.io.json import json_normalize
from datetime import datetime, date, time

In [85]:
cwd = os.getcwd()
print cwd

/Users/jpy/Dropbox/_FLASK/__JUPYTER__/OCF


In [86]:
pp = pprint.PrettyPrinter(indent=2)

#### common functions

In [87]:
def findFromPathString(path_string, JSON, separator="/"):     
  paths = path_string.split(separator)
  data = JSON
  for i in range(0,len(paths)):
    data = data[paths[i]]
  return data

In [88]:
def flatten_json(y):
    out = {}

    def flatten(x, name=''):
        if type(x) is dict:
            for a in x:
                flatten(x[a], name + a + '_')
        elif type(x) is list:
            i = 0
            for a in x:
                flatten(a, name + str(i) + '_')
                i += 1
        else:
            out[name[:-1]] = x

    flatten(y)
    return out

#### settings

19243
[0, 1, 2]


In [90]:

root_url_corpos = "http://opencorporatefacts.fr/api/corporates"
# params_corpos = { 'page' : 1 , 'id' : None , 'CompanyNumber' : None }
results_path_corpos = "hydra:member"
output_filename_corpos = "corpos"

root_url_accounts = "http://opencorporatefacts.fr/api/compte_de_resultats"
# params_accounts = { 'page' : 1, 'Corporate' : None }
results_path_accounts = "hydra:member"
output_filename_accounts = "accounts"

root_url_companies = "http://api.cquest.org/company/" ### pass CompanyNumber after root_url
output_filename_companies = "companies"


#### tests

In [91]:
### test requests
resp = requests.get( root_url_corpos, params = { 'page' : 1 } ).json()
results = findFromPathString( results_path_corpos, resp )

In [92]:
print pp.pprint(results)

[ { u'@id': u'/api/corporates/2',
    u'@type': u'Corporate',
    u'CompanyNumber': u'414119735',
    u'ComptesDeResultats': [u'/api/compte_de_resultats/1'],
    u'CorporateShareholders': [],
    u'IndustryCode': u'',
    u'Name': u'AYMING',
    u'OpenCorporateURL': u'https://opencorporates.com/companies/fr/414119735',
    u'documentDeReferences': [],
    u'id': 2,
    u'shareholdings': []},
  { u'@id': u'/api/corporates/3',
    u'@type': u'Corporate',
    u'CompanyNumber': u'419373055',
    u'ComptesDeResultats': [ u'/api/compte_de_resultats/7713',
                             u'/api/compte_de_resultats/2'],
    u'CorporateShareholders': [],
    u'IndustryCode': u'',
    u'Name': u'NORTH SOUTH CONNECTIONS FRANCE',
    u'OpenCorporateURL': u'https://opencorporates.com/companies/fr/419373055',
    u'documentDeReferences': [],
    u'id': 3,
    u'shareholdings': []},
  { u'@id': u'/api/corporates/4',
    u'@type': u'Corporate',
    u'CompanyNumber': u'329936793',
    u'ComptesDeResultats

In [93]:
for i in results : 
    print i
    print

{u'OpenCorporateURL': u'https://opencorporates.com/companies/fr/414119735', u'Name': u'AYMING', u'shareholdings': [], u'CorporateShareholders': [], u'ComptesDeResultats': [u'/api/compte_de_resultats/1'], u'documentDeReferences': [], u'IndustryCode': u'', u'id': 2, u'@id': u'/api/corporates/2', u'@type': u'Corporate', u'CompanyNumber': u'414119735'}

{u'OpenCorporateURL': u'https://opencorporates.com/companies/fr/419373055', u'Name': u'NORTH SOUTH CONNECTIONS FRANCE', u'shareholdings': [], u'CorporateShareholders': [], u'ComptesDeResultats': [u'/api/compte_de_resultats/7713', u'/api/compte_de_resultats/2'], u'documentDeReferences': [], u'IndustryCode': u'', u'id': 3, u'@id': u'/api/corporates/3', u'@type': u'Corporate', u'CompanyNumber': u'419373055'}

{u'OpenCorporateURL': u'https://opencorporates.com/companies/fr/329936793', u'Name': u'MECAHERS GROUP', u'shareholdings': [], u'CorporateShareholders': [], u'ComptesDeResultats': [u'/api/compte_de_resultats/3'], u'documentDeReferences': [

{u'OpenCorporateURL': u'https://opencorporates.com/companies/fr/525046801', u'Name': u'PETER & WOLFS', u'shareholdings': [], u'CorporateShareholders': [], u'ComptesDeResultats': [], u'documentDeReferences': [], u'IndustryCode': u'7022Z', u'id': 28, u'@id': u'/api/corporates/28', u'@type': u'Corporate', u'CompanyNumber': u'525046801'}

{u'OpenCorporateURL': u'https://opencorporates.com/companies/fr/388830895', u'Name': u'LA SAISONNERAIE', u'shareholdings': [], u'CorporateShareholders': [], u'ComptesDeResultats': [u'/api/compte_de_resultats/13'], u'documentDeReferences': [], u'IndustryCode': u'8710A', u'id': 29, u'@id': u'/api/corporates/29', u'@type': u'Corporate', u'CompanyNumber': u'388830895'}

{u'OpenCorporateURL': u'https://opencorporates.com/companies/fr/817942956', u'Name': u'CORNOUAILLE RENOVATION DOMMAGE OUVRAGE - CRDO', u'shareholdings': [], u'CorporateShareholders': [], u'ComptesDeResultats': [u'/api/compte_de_resultats/14'], u'documentDeReferences': [], u'IndustryCode': u'43

#### main requests functions

In [ ]:
def get_req_results ( url, pages_range, number_of_pages, results_path, output_filename, debug=False, out_formats=["csv"] ) : 
    ''' 
    retrieve results from requests 
    and write csv 
    works with API platform only for now
    '''
    
    start_time = datetime.now()
    
    ### output file
    if "csv" in out_formats :
        file_data_csv = cwd + '/' + output_filename + '.csv'
    if "json" in out_formats : 
        file_data_json = cwd + '/' + output_filename + '.json'
    
    ### count pages
    count = 0
    
    ### requests and writing file
    
    for page in pages_range : 
        
        if page > 0 : 
            
            print "\n page number : {} / {}".format(page, number_of_pages)
            
            resp = requests.get( url, params = { 'page' : page } ).json()
            results = findFromPathString( results_path, resp )
            
            page_time = datetime.now()
            delta_time = page_time - start_time
            print "delta_time (after request) : {} ".format(delta_time)

            # loop results
            for data in results : 
                
                if debug : 
                    # print pp.pprint(data) 
                    print data
                    print
                data_flat = flatten_json(data)
                # print data_flat
                
                ### CSV
                if "csv" in out_formats : 
                    
                    ### creates headers
                    if count == 0:
                        with open(file_data_csv, 'w') as csv_file :
                            csv_writer = csv.writer(csv_file, delimiter=';')
                            header = data.keys()
                            csv_writer.writerow(header)
                            count += 1
                    else : 
                        with open(file_data_csv, 'a') as csv_file :
                            csv_writer = csv.writer(csv_file, delimiter=';')
                            csv_writer.writerow( data.values() )
                            count += 1

                        
                ### JSON
                if "json" in out_formats : 
                    if count == 0 :
                        with open(file_data_json, 'w') as json_file :
                            json_file.write(json.dumps( [data] ).encode())         #If empty, write an array
                            count += 1
                    else : 
                        with open(file_data_json, 'ab+') as json_file :
                            json_file.seek(-1,2)           
                            json_file.truncate()                           #Remove the last character, open the array
                            json_file.write(' , '.encode())                #Write the separator
                            json_file.write(json.dumps(data).encode())     #Dump the dictionary
                            json_file.write(']'.encode())                  #Close the array
                            count += 1


#### getting data

In [122]:
### TEST / get corpos 
number_of_pages_corpos_test = 3
pages_range_test = range(number_of_pages_corpos)
get_req_results( 
    root_url_corpos, 
    pages_range_test, 
    number_of_pages_corpos_test,
    results_path_corpos, 
    csv_output_corpos, 
    debug=True, 
    out_formats=["json"] 
) 


 page number : 1 

{u'OpenCorporateURL': u'https://opencorporates.com/companies/fr/414119735', u'Name': u'AYMING', u'shareholdings': [], u'CorporateShareholders': [], u'ComptesDeResultats': [u'/api/compte_de_resultats/1'], u'documentDeReferences': [], u'IndustryCode': u'', u'id': 2, u'@id': u'/api/corporates/2', u'@type': u'Corporate', u'CompanyNumber': u'414119735'}

{u'OpenCorporateURL': u'https://opencorporates.com/companies/fr/419373055', u'Name': u'NORTH SOUTH CONNECTIONS FRANCE', u'shareholdings': [], u'CorporateShareholders': [], u'ComptesDeResultats': [u'/api/compte_de_resultats/7713', u'/api/compte_de_resultats/2'], u'documentDeReferences': [], u'IndustryCode': u'', u'id': 3, u'@id': u'/api/corporates/3', u'@type': u'Corporate', u'CompanyNumber': u'419373055'}

{u'OpenCorporateURL': u'https://opencorporates.com/companies/fr/329936793', u'Name': u'MECAHERS GROUP', u'shareholdings': [], u'CorporateShareholders': [], u'ComptesDeResultats': [u'/api/compte_de_resultats/3'], u'docu

{u'OpenCorporateURL': u'https://opencorporates.com/companies/fr/347531402', u'Name': u'AMALLY', u'shareholdings': [], u'CorporateShareholders': [], u'ComptesDeResultats': [], u'documentDeReferences': [], u'IndustryCode': u'4730Z', u'id': 32, u'@id': u'/api/corporates/32', u'@type': u'Corporate', u'CompanyNumber': u'347531402'}

{u'OpenCorporateURL': u'https://opencorporates.com/companies/fr/490848223', u'Name': u'LES CONSTRUCTEURS ASSOCIES', u'shareholdings': [], u'CorporateShareholders': [], u'ComptesDeResultats': [], u'documentDeReferences': [], u'IndustryCode': u'4399C', u'id': 33, u'@id': u'/api/corporates/33', u'@type': u'Corporate', u'CompanyNumber': u'490848223'}

{u'OpenCorporateURL': u'https://opencorporates.com/companies/fr/481317212', u'Name': u'AGEN RENOVATION', u'shareholdings': [], u'CorporateShareholders': [], u'ComptesDeResultats': [u'/api/compte_de_resultats/5629'], u'documentDeReferences': [], u'IndustryCode': u'6810Z', u'id': 34, u'@id': u'/api/corporates/34', u'@typ

{u'OpenCorporateURL': u'https://opencorporates.com/companies/fr/518552435', u'Name': u'G ENERGY', u'shareholdings': [], u'CorporateShareholders': [], u'ComptesDeResultats': [u'/api/compte_de_resultats/22'], u'documentDeReferences': [], u'IndustryCode': u'7112B', u'id': 62, u'@id': u'/api/corporates/62', u'@type': u'Corporate', u'CompanyNumber': u'518552435'}

{u'OpenCorporateURL': u'https://opencorporates.com/companies/fr/490902632', u'Name': u'JLC', u'shareholdings': [], u'CorporateShareholders': [], u'ComptesDeResultats': [], u'documentDeReferences': [], u'IndustryCode': u'6420Z', u'id': 63, u'@id': u'/api/corporates/63', u'@type': u'Corporate', u'CompanyNumber': u'490902632'}

{u'OpenCorporateURL': u'https://opencorporates.com/companies/fr/442238002', u'Name': u'CHAMP LIBRE PRODUCTION', u'shareholdings': [], u'CorporateShareholders': [], u'ComptesDeResultats': [], u'documentDeReferences': [], u'IndustryCode': u'9001Z', u'id': 64, u'@id': u'/api/corporates/64', u'@type': u'Corporate'

{u'OpenCorporateURL': u'https://opencorporates.com/companies/fr/794462770', u'Name': u'HOLDING MJJ', u'shareholdings': [], u'CorporateShareholders': [], u'ComptesDeResultats': [u'/api/compte_de_resultats/5636', u'/api/compte_de_resultats/214521', u'/api/compte_de_resultats/214520'], u'documentDeReferences': [], u'IndustryCode': u'6420Z', u'id': 92, u'@id': u'/api/corporates/92', u'@type': u'Corporate', u'CompanyNumber': u'794462770'}

{u'OpenCorporateURL': u'https://opencorporates.com/companies/fr/513939249', u'Name': u'SUBEST-MEDIALOGIE', u'shareholdings': [], u'CorporateShareholders': [], u'ComptesDeResultats': [], u'documentDeReferences': [], u'IndustryCode': u'5610C', u'id': 93, u'@id': u'/api/corporates/93', u'@type': u'Corporate', u'CompanyNumber': u'513939249'}

{u'OpenCorporateURL': u'https://opencorporates.com/companies/fr/534278882', u'Name': u'DUFOUR', u'shareholdings': [], u'CorporateShareholders': [], u'ComptesDeResultats': [], u'documentDeReferences': [], u'IndustryCode':

In [ ]:
### get corpos data
number_of_results_corpos = 577297
results_per_page = 30
number_of_pages_corpos = int( math.ceil( number_of_results_corpos / results_per_page ) )
print number_of_pages_corpos
pages_to_scrap_corpos = range( number_of_pages_corpos )
print pages_to_scrap_corpos[:3]

get_req_results( 
    root_url_corpos, 
    pages_to_scrap_corpos, 
    number_of_pages_corpos,
    results_path_corpos, 
    output_filename_corpos, 
    debug=False, 
    out_formats=["json"] 
) 


 page number : 1 / 19243 


 page number : 2 / 19243 


 page number : 3 / 19243 


 page number : 4 / 19243 


 page number : 5 / 19243 


 page number : 6 / 19243 


 page number : 7 / 19243 


 page number : 8 / 19243 


 page number : 9 / 19243 


 page number : 10 / 19243 


 page number : 11 / 19243 


 page number : 12 / 19243 


 page number : 13 / 19243 


 page number : 14 / 19243 


 page number : 15 / 19243 


 page number : 16 / 19243 


 page number : 17 / 19243 


 page number : 18 / 19243 


 page number : 19 / 19243 


 page number : 20 / 19243 


 page number : 21 / 19243 


 page number : 22 / 19243 


 page number : 23 / 19243 


 page number : 24 / 19243 


 page number : 25 / 19243 


 page number : 26 / 19243 


 page number : 27 / 19243 


 page number : 28 / 19243 


 page number : 29 / 19243 


 page number : 30 / 19243 


 page number : 31 / 19243 


 page number : 32 / 19243 


 page number : 33 / 19243 


 page number : 34 / 19243 


 page number : 35 / 19


 page number : 278 / 19243 


 page number : 279 / 19243 


 page number : 280 / 19243 


 page number : 281 / 19243 


 page number : 282 / 19243 


 page number : 283 / 19243 


 page number : 284 / 19243 


 page number : 285 / 19243 


 page number : 286 / 19243 


 page number : 287 / 19243 


 page number : 288 / 19243 


 page number : 289 / 19243 


 page number : 290 / 19243 


 page number : 291 / 19243 


 page number : 292 / 19243 


 page number : 293 / 19243 


 page number : 294 / 19243 


 page number : 295 / 19243 


 page number : 296 / 19243 


 page number : 297 / 19243 


 page number : 298 / 19243 


 page number : 299 / 19243 


 page number : 300 / 19243 


 page number : 301 / 19243 


 page number : 302 / 19243 


 page number : 303 / 19243 


 page number : 304 / 19243 


 page number : 305 / 19243 


 page number : 306 / 19243 


 page number : 307 / 19243 


 page number : 308 / 19243 


 page number : 309 / 19243 


 page number : 310 / 19243 


 page num


 page number : 552 / 19243 


 page number : 553 / 19243 


 page number : 554 / 19243 


 page number : 555 / 19243 


 page number : 556 / 19243 


 page number : 557 / 19243 


 page number : 558 / 19243 


 page number : 559 / 19243 


 page number : 560 / 19243 


 page number : 561 / 19243 


 page number : 562 / 19243 


 page number : 563 / 19243 


 page number : 564 / 19243 


 page number : 565 / 19243 


 page number : 566 / 19243 


 page number : 567 / 19243 


 page number : 568 / 19243 


 page number : 569 / 19243 


 page number : 570 / 19243 


 page number : 571 / 19243 


 page number : 572 / 19243 


 page number : 573 / 19243 


 page number : 574 / 19243 


 page number : 575 / 19243 


 page number : 576 / 19243 


 page number : 577 / 19243 


 page number : 578 / 19243 


 page number : 579 / 19243 


 page number : 580 / 19243 


 page number : 581 / 19243 


 page number : 582 / 19243 


 page number : 583 / 19243 


 page number : 584 / 19243 


 page num


 page number : 826 / 19243 


 page number : 827 / 19243 


 page number : 828 / 19243 


 page number : 829 / 19243 


 page number : 830 / 19243 


 page number : 831 / 19243 


 page number : 832 / 19243 


 page number : 833 / 19243 


 page number : 834 / 19243 


 page number : 835 / 19243 


 page number : 836 / 19243 


 page number : 837 / 19243 


 page number : 838 / 19243 


 page number : 839 / 19243 


 page number : 840 / 19243 


 page number : 841 / 19243 


 page number : 842 / 19243 


 page number : 843 / 19243 


 page number : 844 / 19243 


 page number : 845 / 19243 


 page number : 846 / 19243 


 page number : 847 / 19243 


 page number : 848 / 19243 


 page number : 849 / 19243 


 page number : 850 / 19243 


 page number : 851 / 19243 


 page number : 852 / 19243 


 page number : 853 / 19243 


 page number : 854 / 19243 


 page number : 855 / 19243 


 page number : 856 / 19243 


 page number : 857 / 19243 


 page number : 858 / 19243 


 page num


 page number : 1096 / 19243 


 page number : 1097 / 19243 


 page number : 1098 / 19243 


 page number : 1099 / 19243 


 page number : 1100 / 19243 


 page number : 1101 / 19243 


 page number : 1102 / 19243 


 page number : 1103 / 19243 


 page number : 1104 / 19243 


 page number : 1105 / 19243 


 page number : 1106 / 19243 


 page number : 1107 / 19243 


 page number : 1108 / 19243 


 page number : 1109 / 19243 


 page number : 1110 / 19243 


 page number : 1111 / 19243 


 page number : 1112 / 19243 


 page number : 1113 / 19243 


 page number : 1114 / 19243 


 page number : 1115 / 19243 


 page number : 1116 / 19243 


 page number : 1117 / 19243 


 page number : 1118 / 19243 


 page number : 1119 / 19243 


 page number : 1120 / 19243 


 page number : 1121 / 19243 


 page number : 1122 / 19243 


 page number : 1123 / 19243 


 page number : 1124 / 19243 


 page number : 1125 / 19243 


 page number : 1126 / 19243 


 page number : 1127 / 19243 


 page n

...


 page number : 3746 / 19243 


 page number : 3747 / 19243 


 page number : 3748 / 19243 


 page number : 3749 / 19243 


 page number : 3750 / 19243 


 page number : 3751 / 19243 


 page number : 3752 / 19243 


 page number : 3753 / 19243 


 page number : 3754 / 19243 


 page number : 3755 / 19243 


 page number : 3756 / 19243 


 page number : 3757 / 19243 


 page number : 3758 / 19243 


 page number : 3759 / 19243 


 page number : 3760 / 19243 


 page number : 3761 / 19243 


 page number : 3762 / 19243 


 page number : 3763 / 19243 


 page number : 3764 / 19243 


 page number : 3765 / 19243 


 page number : 3766 / 19243 


 page number : 3767 / 19243 


 page number : 3768 / 19243 


 page number : 3769 / 19243 


 page number : 3770 / 19243 


 page number : 3771 / 19243 


 page number : 3772 / 19243 


 page number : 3773 / 19243 


 page number : 3774 / 19243 


 page number : 3775 / 19243 


 page number : 3776 / 19243 


 page number : 3777 / 19243 


 page n


 page number : 4011 / 19243 


 page number : 4012 / 19243 


 page number : 4013 / 19243 


 page number : 4014 / 19243 


 page number : 4015 / 19243 


 page number : 4016 / 19243 


 page number : 4017 / 19243 


 page number : 4018 / 19243 


 page number : 4019 / 19243 


 page number : 4020 / 19243 


 page number : 4021 / 19243 


 page number : 4022 / 19243 


 page number : 4023 / 19243 


 page number : 4024 / 19243 


 page number : 4025 / 19243 


 page number : 4026 / 19243 


 page number : 4027 / 19243 


 page number : 4028 / 19243 


 page number : 4029 / 19243 


 page number : 4030 / 19243 


 page number : 4031 / 19243 


 page number : 4032 / 19243 


 page number : 4033 / 19243 


 page number : 4034 / 19243 


 page number : 4035 / 19243 


 page number : 4036 / 19243 


 page number : 4037 / 19243 


 page number : 4038 / 19243 


 page number : 4039 / 19243 


 page number : 4040 / 19243 


 page number : 4041 / 19243 


 page number : 4042 / 19243 


 page n


 page number : 4276 / 19243 


 page number : 4277 / 19243 


 page number : 4278 / 19243 


 page number : 4279 / 19243 


 page number : 4280 / 19243 


 page number : 4281 / 19243 


 page number : 4282 / 19243 


 page number : 4283 / 19243 


 page number : 4284 / 19243 


 page number : 4285 / 19243 


 page number : 4286 / 19243 


 page number : 4287 / 19243 


 page number : 4288 / 19243 


 page number : 4289 / 19243 


 page number : 4290 / 19243 


 page number : 4291 / 19243 


 page number : 4292 / 19243 


 page number : 4293 / 19243 


 page number : 4294 / 19243 


 page number : 4295 / 19243 


 page number : 4296 / 19243 


 page number : 4297 / 19243 


 page number : 4298 / 19243 


 page number : 4299 / 19243 


 page number : 4300 / 19243 


 page number : 4301 / 19243 


 page number : 4302 / 19243 


 page number : 4303 / 19243 


 page number : 4304 / 19243 


 page number : 4305 / 19243 


 page number : 4306 / 19243 


 page number : 4307 / 19243 


 page n


 page number : 4541 / 19243 


 page number : 4542 / 19243 


 page number : 4543 / 19243 


 page number : 4544 / 19243 


 page number : 4545 / 19243 


 page number : 4546 / 19243 


 page number : 4547 / 19243 


 page number : 4548 / 19243 


 page number : 4549 / 19243 


 page number : 4550 / 19243 


 page number : 4551 / 19243 


 page number : 4552 / 19243 


 page number : 4553 / 19243 


 page number : 4554 / 19243 


 page number : 4555 / 19243 


 page number : 4556 / 19243 


 page number : 4557 / 19243 


 page number : 4558 / 19243 


 page number : 4559 / 19243 


 page number : 4560 / 19243 


 page number : 4561 / 19243 


 page number : 4562 / 19243 


 page number : 4563 / 19243 


 page number : 4564 / 19243 


 page number : 4565 / 19243 


 page number : 4566 / 19243 


 page number : 4567 / 19243 


 page number : 4568 / 19243 


 page number : 4569 / 19243 


 page number : 4570 / 19243 


 page number : 4571 / 19243 


 page number : 4572 / 19243 


 page n


 page number : 4806 / 19243 


 page number : 4807 / 19243 


 page number : 4808 / 19243 


 page number : 4809 / 19243 


 page number : 4810 / 19243 


 page number : 4811 / 19243 


 page number : 4812 / 19243 


 page number : 4813 / 19243 


 page number : 4814 / 19243 


 page number : 4815 / 19243 


 page number : 4816 / 19243 


 page number : 4817 / 19243 


 page number : 4818 / 19243 


 page number : 4819 / 19243 


 page number : 4820 / 19243 


 page number : 4821 / 19243 


 page number : 4822 / 19243 


 page number : 4823 / 19243 


 page number : 4824 / 19243 


 page number : 4825 / 19243 


 page number : 4826 / 19243 


 page number : 4827 / 19243 


 page number : 4828 / 19243 


 page number : 4829 / 19243 


 page number : 4830 / 19243 


 page number : 4831 / 19243 


 page number : 4832 / 19243 


 page number : 4833 / 19243 


 page number : 4834 / 19243 


 page number : 4835 / 19243 


 page number : 4836 / 19243 


 page number : 4837 / 19243 


 page n

In [ ]:
### get accounts data
number_of_results_account = 316182
results_per_page = 30
number_of_pages_accounts = int( math.ceil( number_of_results_account / results_per_page ) )
print number_of_pages_accounts
pages_to_scrap_accounts = range( number_of_pages_accounts )
print pages_to_scrap_accounts[:3]

get_req_results( 
    root_url_accounts, 
    pages_to_scrap_accounts, 
    number_of_pages_accounts,
    results_path_accounts, 
    output_filename_accounts, 
    debug=False, 
    out_formats=["json"] 
) 